# Setup

## Instructions

This notebook utilizes the Kaggle API. Follow these steps in order to get the necessary credentials to continue:

1. Sign up for a Kaggle account at https://www.kaggle.com.
2. Go to the 'Account' tab of your user profile - ```https://www.kaggle.com/{username}/account```.
3. Select 'Create New API Token' under 'API' section.
    - This will trigger the download of kaggle.json, a file containing your API credentials. 
4. Place this file in the location:
    - ~/.kaggle/kaggle.json (for macOS/unix)
    - C:/Users/username/.kaggle/kaggle.json (for Windows) 
    - You can check the exact location, sans drive, with echo %HOMEPATH%). 
    - You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to:
        - $KAGGLE_CONFIG_DIR/kaggle.json (for macOS/unix)
        - %KAGGLE_CONFIG_DIR%\kaggle.json (for Windows)

Note: this notebook uses functions written in Python to query the Kaggle API. **This code will only work for python 3.7 or later**.

## Additional Information

Documentation Guide:
- Kaggle API ([Kaggle](https://www.kaggle.com/docs/api))
- Kaggle API ([GitHub](https://github.com/Kaggle/kaggle-api)) 

## Workflow overview

<img src="../images/Kaggle_workflow.jpg" width=650 height=650 align="left"/>

## Import libraries

In [1]:
# Allow system to search parent folder for local imports
import sys
sys.path.append('..')

import kaggle
import subprocess # Used to run unix commands
import pandas as pd # For storing/manipulating command data
from io import StringIO # Lets us read csv string output from command into DataFrame
import json # Reading back the metadata files
from tqdm import tqdm # Gives status bar on loop completion
import itertools # For efficient looping over queries
import os # Exporting saved results
from collections import OrderedDict
from utils import flatten_nested_df
from flatten_json import flatten

## Query #1: query API based on search terms and search types

Function `get_all_search_outputs` queries the Kaggle API for all combinations of search terms and search types specified and returns the results as a dictionary of dataframes (one dataframe for each query combination)
- Calls function `get_individual_search_output`

In [2]:
def get_all_search_outputs(search_terms, search_types, flatten_output=False):
    """
    Call the Kaggle API for each search term and search type. 
    Results are retured in results['{term}_{type}'] = df
    
    Params:
    - search_terms (list-like): collection of search terms to query over
    - search_types (list-like): collection of search types to query over
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - results (dict): dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    num_searches = len(search_terms) * len(search_types)
    results = OrderedDict()
    
    for search_term, search_type in itertools.product(search_terms, search_types):
        results[(search_term, search_type)] = get_individual_search_output(search_term, search_type, flatten_output)
        
    return results

Function `get_individual_search_output` queries the Kaggle API with the specified search term (e.g., “machine learning”) and search type (must be either “datasets” or “kernels”)
- Searches across all returned pages
- Calls function `_convert_string_csv_output_to_dataframe` which converts results from API (strings in semi-structured table) to dataframe format
- Result is a dataframe (one dataframe per search term/search type combination)

In [3]:
def get_individual_search_output(search_term, search_type, flatten_output=False):
    """
    Calls the Kaggle API with the specified search term and returns the search output results.
    
    Params:
    - search_term (str): keyword to seach for
    - search_type (str): objects to search over (must be either datasets or kernels)
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - df (pandas.DataFrame): DataFrame containing the output of the search query
    """
    
    # Make sure our input is valid
    assert isinstance(search_type, str), 'Search term must be a string'
    assert search_type in ('datasets', 'kernels'), 'Search can only be conducted over datasets or kernels'
    
    # Set search variables
    page_idx = 1
    search_output = ''
    cumulative_output = ''
    completion_phrase = f'No {search_type} found\n'
    
    # Pulls the records for a single page of datasets for the given search term
    # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
    search_output = subprocess.run(['kaggle', search_type, 'list', '-v', 
                                     '-p', str(page_idx),
                                     '-s', search_term], 
                                    capture_output=True).stdout.decode()

    # Once we no longer see new output, we stop
    while search_output != completion_phrase:
        # Accumulate the output
        cumulative_output = cumulative_output + search_output

        # Increments the page count for searching
        page_idx += 1

        # Pulls the records for a single page of datasets for the given search term
        # Runs the command, captures the output in stdout, reads it from stdout, and decodes it to str from binary
        search_output = subprocess.run(['kaggle', search_type, 'list', '-v',
                                         '-p', str(page_idx),
                                         '-s', search_term], 
                                        capture_output=True).stdout.decode()

        # Remove header row
        if search_output != completion_phrase:
            search_output = '\r\n'.join(search_output.split('\r\n')[1::])
    
    # Convert search output to DataFrame
    search_df = _convert_string_csv_output_to_dataframe(cumulative_output)
    
    # Rename columns to match names present in metadata df
    search_df = search_df.rename(columns={'ref': 'id', 
                                          'downloadCount': 'totalDownloads', 
                                          'voteCount': 'totalVotes'}
                                )
        
    if flatten_output:
        search_df = flatten_nested_df(search_df)
        
    # Need to change dtypes for mergin
    search_df = search_df.convert_dtypes()
    
    return search_df.iloc[:50]

Function `_convert_string_csv_output_to_dataframe` converts raw results from API (strings in semi-structured table) to dataframe format

In [4]:
def _convert_string_csv_output_to_dataframe(output):
    """
    Given a string variable in csv format, returns a Pandas DataFrame
    
    Params:
    - output (str): csv-styled string to be converted
    
    Returns:
    - df (pandas.DataFrame): DataFrame consisting of data from 'output' string variable
    """
    
    # Create DataFrame of results
    output = StringIO(output)
    df = pd.read_csv(output)
    
    return df

#### Run query #1 functions - example

In [5]:
search_terms = ['"machine learning"', '"artificial intelligence"']
search_types = ['datasets', 'kernels']

In [6]:
search_output_dict = get_all_search_outputs(search_terms, search_types, flatten_output=True)

In [7]:
sample_key = (search_terms[0], search_types[0])
sample_df = search_output_dict[sample_key]

In [8]:
sample_df

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating
0,kaggle/kaggle-survey-2018,2018 Kaggle Machine Learning & Data Science Su...,4MB,2018-11-03 22:35:07,15381,976,0.85294116
1,kaggle/kaggle-survey-2017,2017 Kaggle Machine Learning & Data Science Su...,4MB,2017-10-27 22:03:03,22997,824,0.8235294
2,alopez247/pokemon,Pokémon for Data Mining and Machine Learning,715KB,2017-03-05 15:01:26,10470,241,0.85294116
3,kashnitsky/mlcourse,mlcourse.ai,51MB,2018-12-09 16:45:09,25404,1376,0.88235295
4,kaushil268/disease-prediction-using-machine-le...,Disease Prediction Using Machine Learning,30KB,2020-05-15 03:58:44,2413,42,0.8235294
5,henrysue/online-shoppers-intention,Online Shoppers Intention UCI Machine Learning,252KB,2020-01-15 20:19:02,1224,33,1.0
6,shashwatwork/phishing-dataset-for-machine-lear...,Phishing Dataset for Machine Learning,234KB,2021-05-28 14:47:37,302,20,0.9411765
7,rikdifos/credit-card-approval-prediction,Credit Card Approval Prediction,5MB,2020-03-24 10:04:48,15583,319,1.0
8,cristeaioan/ffml-dataset,Food For Machine Learning,64MB,2020-04-24 09:10:45,723,18,0.8125
9,saxenapriyansh/coursera-machine-learning-andre...,Coursera - Machine Learning - Andrew_Ng,4KB,2018-12-25 12:27:33,423,19,0.875


## Query #2: query API for full metadata for hits from initial query

Note: Unable to find a way to store metadata in memory as opposed to saving file, but this workaround appears to be functional.

Function `_retrieve_object_json` uses the path for each object (each dataframe in result #1 ordered dictionary to query API for metadata associated with each object. Loads JSON results as dictionary.

In [9]:
def _retrieve_object_json(object_path, flatten_output=False):
    """
    Queries Kaggle for metadata json file & returns the json data as a dictionary
    
    Params:
    - object_path (str): path for the dataset
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - metadata_dict (dict): dictionary containing json metadata
    """
    
    # Download the metadata
    process = subprocess.run(['kaggle', 'datasets', 'metadata', object_path])
    
    # If the query did not return metadata
    if process.returncode == 1:
        return None
    else:
        # Access the metadata and load it in as a dictionary
        with open('dataset-metadata.json') as file:
            json_data = json.load(file)

        if flatten_output:
            json_data = flatten(json_data)

        return json_data

Function `get_query_metadata` extracts metadata associated with each object and formats as dataframe
- Calls function `_retrieve_object_json`
- For each object, appends JSON results from `_retrieve_object_json` to a dataframe (converting to dataframe format)
- Output is single dataframe for each search query (matching each dataframe in result #1 ordered dictionary)

In [10]:
def get_query_metadata(object_paths, flatten_output=False):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths (str/list-like): string or list of strings containing the paths for the objects
    - flatten_output (bool): optional (default=False)
    
    Returns:
    - metadata_df (pandas.DataFrame): DataFrame containing metadata for the requested datasets
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
        
    # Run first query
    json_data = _retrieve_object_json(object_paths[0], flatten_output)
        
    # Create DataFrame to store metadata in, using columns found in first query, and then add query info
    metadata_df = pd.DataFrame(json_data)
        
    # Pulls metadata information for each dataset found above
    for object_path in tqdm(object_paths[1:50:]):
        # Download & load the metadata
        json_data = _retrieve_object_json(object_path, flatten_output)

        # Store the metadata into our DataFrame created above
        metadata_df = metadata_df.append(json_data, ignore_index=True)
        
    # Convert datatypes (for some reason totalDownloads and totalViews are floats)
    metadata_df = metadata_df.convert_dtypes()
        
    return metadata_df

Function `get_all_metadata` uses a `for` loop to put dataframes into an ordered dictionary, matching result #1 ordered dictionary
- Calls function `get_query_metadata`

In [11]:
def get_all_metadata(search_output_dict, flatten_output=False):
    """
    Retrieves all of the metadata that relates to the provided DataFrames
    
    Params:
    - search_output_dict : dict
        Dictionary of DataFrames from get_all_search_outputs
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output  
      
    Returns:
    - metadata_dict : collections.OrderedDict
        OrderedDict of DataFrames with metadata for each query
        Order matches the order of search_output_dict
    """

    ## Extract IDs from DataFrame, and returns as list of strings
    metadata_dict = OrderedDict()

    for query, df in search_output_dict.items():
        # Kernels do not have metadata
        if 'kernels' not in query:
            print(f'Retrieving {query} metadata')
            # Create object paths
            _, search_type = query
            object_paths = df.id.values

            metadata_dict[query] = get_query_metadata(object_paths, flatten_output)
        
    return metadata_dict

In [12]:
metadata_dict = get_all_metadata(search_output_dict, flatten_output=True)

Retrieving ('"machine learning"', 'datasets') metadata


100%|██████████| 49/49 [00:32<00:00,  1.53it/s]


Retrieving ('"artificial intelligence"', 'datasets') metadata


100%|██████████| 49/49 [00:59<00:00,  1.21s/it]


## Combine results of query #1 and query #2

Function `merge_search_and_metadata_dicts` merges the output dictionaries from query #1 and query #2 to a single ordered dictionary and (optional) saves the results as a single csv file

In [13]:
def merge_search_and_metadata_dicts(search_dict, metadata_dict, on=None, left_on=None, right_on=None, save=False):
    """
    Merges together all of the search and metadata DataFrames by the given 'on' key
    
    Params:
    - search_dict (dict): dictionary of search output results
    - metadata_dict (dict): dictionary of metadata results
    - on (str/list-like): column name(s) to merge the two dicts on
    - left_on (str/list-like): column name(s) to merge the left dict on
    - right_on (str/list-like): column name(s) to merge the right dict on
    - save=False, optional (bool/list-like): specifies if the output DataFrames should be saved
        If True: saves to file of format 'data/kaggle/kaggle_{search_term}_{search_type}.csv'
        If list-like: saves to respective location in list of save locations
            Must contain enough strings (one per query; len(search_terms) * len(search_types))
            
    Returns:
    - df_dict (OrderedDict): OrderedDict containing all of the merged search/metadata dicts
    """
    
    num_dataframes = len(search_dict)
    
    # Ensure the save variable data is proper
    try:
        if isinstance(save, bool):
            save = [save] * num_dataframes
        assert len(save) == num_dataframes
    except:
        raise ValueError('Incorrect save value(s)')

    # Merge the DataFrames
    df_dict = OrderedDict()
    for query_key, save_loc in zip(search_dict.keys(), save):
        search_df = search_dict[query_key]
        if query_key in metadata_dict:
            # Merge small version of "full" dataframe with "detailed" dataframe
            metadata_df = metadata_dict[query_key]
            df_all = pd.merge(search_df, metadata_df, on=on, left_on=left_on, right_on=right_on, how='outer')
        else:
            df_all = search_df
        
        # Save DataFrame
        if save_loc:
            data_dir = os.path.join('data', 'kaggle')
            if isinstance(save_loc, str):
                output_file = save_loc
            elif isinstance(save_loc, bool):
                # Ensure kaggle directory is already created
                if not os.path.isdir(data_dir):
                    os.path.mkdir(data_dir)
                
                search_term, search_type = query_key
                output_file = f'{search_term}_{search_type}.csv'
            else:
                raise ValueError('Save type must be bool or str')

            search_df.to_csv(os.path.join(data_dir, output_file), index=False)
        
        df_dict[query_key] = df_all
    
    return df_dict

In [14]:
#run merge function
df_dict = merge_search_and_metadata_dicts(search_output_dict, metadata_dict)

In [15]:
df_dict[sample_key]

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating,collaborators,data,datasetId,...,ownerUser,subtitle,totalViews,keywords_6,keywords_7,collaborators_0_role,collaborators_0_username,keywords,collaborators_1_role,collaborators_1_username
0,kaggle/kaggle-survey-2018,2018 Kaggle Machine Learning & Data Science Su...,4MB,2018-11-03 22:35:07,15381,976,0.85294116,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
1,kaggle/kaggle-survey-2017,2017 Kaggle Machine Learning & Data Science Su...,4MB,2017-10-27 22:03:03,22997,824,0.8235294,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
2,alopez247/pokemon,Pokémon for Data Mining and Machine Learning,715KB,2017-03-05 15:01:26,10470,241,0.85294116,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
3,kashnitsky/mlcourse,mlcourse.ai,51MB,2018-12-09 16:45:09,25404,1376,0.88235295,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
4,kaushil268/disease-prediction-using-machine-le...,Disease Prediction Using Machine Learning,30KB,2020-05-15 03:58:44,2413,42,0.8235294,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,antonyj453/urldataset,Malicious_n_Non-Malicious URL,<NA>,<NA>,2004,31,0.470588,[],[],5423,...,antonyj453,Supervised Machine Learning,14371,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
90,krantiswalke/bank-personal-loan-modelling,Bank_Personal_Loan_Modelling,<NA>,<NA>,847,19,0.647059,[],[],608073,...,krantiswalke,Bank_Personal_Loan_Modelling_(Machine Learning...,6585,naive bayes,svm,<NA>,<NA>,NaN,<NA>,<NA>
91,gabrielegalimberti/movies-example-for-machine-...,Movies example for machine learning activities,<NA>,<NA>,481,12,0.882353,[],[],131297,...,gabrielegalimberti,dataset obtain from other datasets of film wit...,4496,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
92,schirmerchad/bostonhoustingmlnd,Boston Housing,<NA>,<NA>,11724,140,0.823529,[],[],1379,...,schirmerchad,Concerns housing values in suburbs of Boston,69885,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>


#### Example output

In [16]:
#results of query #1
output_df = search_output_dict[sample_key]

#results of query #2
metadata_df = metadata_dict[sample_key]

#result of merging datasets into "full" dataframe
full_df = df_dict[sample_key]

In [17]:
sample_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating
0,kaggle/kaggle-survey-2018,2018 Kaggle Machine Learning & Data Science Su...,4MB,2018-11-03 22:35:07,15381,976,0.85294116
1,kaggle/kaggle-survey-2017,2017 Kaggle Machine Learning & Data Science Su...,4MB,2017-10-27 22:03:03,22997,824,0.8235294
2,alopez247/pokemon,Pokémon for Data Mining and Machine Learning,715KB,2017-03-05 15:01:26,10470,241,0.85294116
3,kashnitsky/mlcourse,mlcourse.ai,51MB,2018-12-09 16:45:09,25404,1376,0.88235295
4,kaushil268/disease-prediction-using-machine-le...,Disease Prediction Using Machine Learning,30KB,2020-05-15 03:58:44,2413,42,0.8235294


In [18]:
metadata_df.head()

,collaborators,data,datasetId,datasetSlug,description,id,id_no,isPrivate,keywords_0,keywords_1,...,totalViews,totalVotes,usabilityRating,keywords_6,keywords_7,collaborators_0_role,collaborators_0_username,keywords,collaborators_1_role,collaborators_1_username
0,[],[],635,pokemon,# Context With the rise of the popularity of...,alopez247/pokemon,635,0,arts and entertainment,games,...,144794,241,0.852941,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
1,[],[],32132,mlcourse,![](https://habrastorage.org/webt/ia/m9/zk/iam...,kashnitsky/mlcourse,32132,0,computer science,data visualization,...,200636,1376,0.882353,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
2,[],[],654897,disease-prediction-using-machine-learning,### Context During the time when Machine Lear...,kaushil268/disease-prediction-using-machine-le...,654897,0,diseases,earth and nature,...,15830,42,0.823529,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
3,[],[],477512,online-shoppers-intention,**Data Set Information:** The dataset consist...,henrysue/online-shoppers-intention,477512,0,universities and colleges,arts and entertainment,...,17301,33,1.0,internet,retail and shopping,<NA>,<NA>,NaN,<NA>,<NA>
4,[],[],1373456,phishing-dataset-for-machine-learning,### Context Anti-phishing refers to efforts t...,shashwatwork/phishing-dataset-for-machine-lear...,1373456,0,research,exploratory data analysis,...,1465,20,0.941176,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>


In [19]:
full_df.head()

,id,title,size,lastUpdated,totalDownloads,totalVotes,usabilityRating,collaborators,data,datasetId,...,ownerUser,subtitle,totalViews,keywords_6,keywords_7,collaborators_0_role,collaborators_0_username,keywords,collaborators_1_role,collaborators_1_username
0,kaggle/kaggle-survey-2018,2018 Kaggle Machine Learning & Data Science Su...,4MB,2018-11-03 22:35:07,15381,976,0.85294116,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
1,kaggle/kaggle-survey-2017,2017 Kaggle Machine Learning & Data Science Su...,4MB,2017-10-27 22:03:03,22997,824,0.8235294,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
2,alopez247/pokemon,Pokémon for Data Mining and Machine Learning,715KB,2017-03-05 15:01:26,10470,241,0.85294116,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
3,kashnitsky/mlcourse,mlcourse.ai,51MB,2018-12-09 16:45:09,25404,1376,0.88235295,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
4,kaushil268/disease-prediction-using-machine-le...,Disease Prediction Using Machine Learning,30KB,2020-05-15 03:58:44,2413,42,0.8235294,NaN,NaN,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
